In [ ]:
import os
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.projects.models import FunctionTool, ToolSet
from datetime import datetime
from legal_agent_tools import user_functions # user functions which can be found in a user_functions.py file.

# Create an Azure AI Client from a connection string, copied from your Azure AI Foundry project.
# It should be in the format "<HostName>;<AzureSubscriptionId>;<ResourceGroup>;<HubName>"
# Customers need to login to Azure subscription via Azure CLI and set the environment variables

project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(),
    conn_str=os.environ["PROJECT_CONNECTION_STRING"],
)

# Initialize agent toolset with user functions
functions = FunctionTool(user_functions)
toolset = ToolSet()
toolset.add(functions)

agent = project_client.agents.create_agent(
    model= os.environ["MODEL_DEPLOYMENT_NAME"], 
    name=f"legal-cases-agent-{datetime.now().strftime('%Y%m%d%H%M')}",
    description="Legal Cases Agent", 
    instructions=f"""
    You are a helpful legal assistant that can retrieve information about legal cases. 
    The current date is {datetime.now().strftime('%Y-%m-%d')}.
    """, 
    toolset=toolset
)
print(f"Created agent, ID: {agent.id}")


Created agent, ID: asst_zwzDCfEmWoIuPDP0gVHM5PAc


In [7]:
# Enable Azure Monitor tracing
from azure.monitor.opentelemetry import configure_azure_monitor

application_insights_connection_string = project_client.telemetry.get_connection_string()
if not application_insights_connection_string:
    print("Application Insights was not enabled for this project.")
    print("Enable it via the 'Tracing' tab in your AI Foundry project page.")
    exit()
configure_azure_monitor(connection_string=application_insights_connection_string)

project_client.telemetry.enable() # Enable Azure Monitor tracing

In [8]:
# Create thread for communication
thread = project_client.agents.create_thread()
print(f"Created thread, ID: {thread.id}")

# Create message to thread
message = project_client.agents.create_message(
    thread_id=thread.id,
    role="user",
    content="Water leaking into the apartment from the floor above, What are the prominent legal precedents in Washington on this problem in the last 25 years?"
)
print(f"Created message, ID: {message.id}")


Created thread, ID: thread_ILOYkJ8mQj8031rJX3kgFdpy
Created message, ID: msg_Cmyg8Care8AaqsGP6tMpNAu5


In [9]:
from pprint import pprint

# Create and process agent run in thread with tools
run = project_client.agents.create_and_process_run(thread_id=thread.id, assistant_id=agent.id)
print(f"Run finished with status: {run.status}")

if run.status == "failed":
    print(f"Run failed: {run.last_error}")

# Fetch and log all messages
messages = project_client.agents.list_messages(thread_id=thread.id)
print(f"Messages: {messages}")
pprint(messages['data'][0]['content'][0]['text']['value'])


Run finished with status: RunStatus.COMPLETED
Messages: {'object': 'list', 'data': [{'id': 'msg_bY3zlEV9WbGmAUqtJkCl0ywa', 'object': 'thread.message', 'created_at': 1741193865, 'assistant_id': 'asst_zwzDCfEmWoIuPDP0gVHM5PAc', 'thread_id': 'thread_ILOYkJ8mQj8031rJX3kgFdpy', 'run_id': 'run_8Ic2huldUvv04zi9XXyy0v3G', 'role': 'assistant', 'content': [{'type': 'text', 'text': {'value': "Here are some prominent legal precedents regarding cases of water leakage or related issues affecting properties in Washington over the past 25 years:\n\n1. **Tucker v. Hayford (2001)**: In this case, tenants claimed damages due to personal injuries from drinking contaminated well water. The court concluded that the tenants could pursue claims under common law, breach of lease agreements, and the Washington Residential Landlord-Tenant Act. The ruling emphasized that landlords have obligations to maintain their properties in a manner that ensures tenant safety and health.\n\n2. **Rothweiler v. Clark County (2

In [5]:
#Delete the agent when done
project_client.agents.delete_agent(agent.id)
print("Deleted agent")

Deleted agent
